In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from openpyxl.utils import get_column_letter
import os

def create_combined_excel(pivot_csv, lookup_csv, output_excel_file):
    # --- Chargement des fichiers CSV ---
    df_pivot = pd.read_csv(pivot_csv, sep=';')
    df_lookup = pd.read_csv(lookup_csv, sep=';')

    # Nettoyage des noms de colonnes et de certaines données
    df_pivot.columns = df_pivot.columns.str.strip()
    df_lookup.columns = df_lookup.columns.str.strip()
    df_lookup['tunnel'] = df_lookup['tunnel'].astype(str).str.strip()
    df_lookup['prod_groupe'] = df_lookup['prod_groupe'].astype(str).str.strip().str.lower()

    # Supprimer les colonnes 'Unnamed' inutiles dans lookup
    df_lookup = df_lookup.drop(columns=[col for col in df_lookup.columns if col.startswith('Unnamed:')])

    # Filtrer les lignes avec prod_groupe vide ou NaN
    df_lookup = df_lookup[(df_lookup['prod_groupe'] != '') & (df_lookup['prod_groupe'] != 'nan')]

    # Création de la colonne NombreEquip : valeur 'nb_equipe' si 'equipements' est vide, sinon vide (à remplir par formule INDEX/MATCH)
    df_lookup['NombreEquip'] = df_lookup.apply(
        lambda row: row['nb_equipe'] if pd.isna(row['equipements']) or str(row['equipements']).strip() == '' else None,
        axis=1
    )

    # Création d'une clé de recherche (LOOKUP_KEY) pour les formules
    df_lookup['lookup_key'] = df_lookup['prod_groupe'].str.lower() + '|' + df_lookup['tunnel']

    # --- Ecriture du fichier Excel ---
    with pd.ExcelWriter(output_excel_file, engine='openpyxl') as writer:
        # Noms des feuilles
        pivot_sheet_name = os.path.splitext(os.path.basename(pivot_csv))[0]
        lookup_sheet_name = 'produits_tunnels'  # renommage de la feuille lookup

        # Ecrire les données dans les feuilles
        df_pivot.to_excel(writer, sheet_name=pivot_sheet_name, index=False)
        df_lookup.to_excel(writer, sheet_name=lookup_sheet_name, index=False)

        workbook = writer.book
        sheet_pivot = workbook[pivot_sheet_name]
        sheet_lookup = workbook[lookup_sheet_name]

        # Colonnes utiles pour formules dans lookup
        equip_col_letter = get_column_letter(df_lookup.columns.get_loc('equipements') + 1)
        tunnel_col_letter = get_column_letter(df_lookup.columns.get_loc('tunnel') + 1)
        prodgroupe_col_letter = get_column_letter(df_lookup.columns.get_loc('prod_groupe') + 1)
        key_col_letter = get_column_letter(df_lookup.columns.get_loc('lookup_key') + 1)
        nombre_equip_col_idx = df_lookup.columns.get_loc('NombreEquip') + 1
        nombre_equip_col_letter = get_column_letter(nombre_equip_col_idx)
        cout_ht_col_idx = df_lookup.columns.get_loc('cout_ht') + 1
        cout_ht_col_letter = get_column_letter(cout_ht_col_idx)
        prod_groupe_col_idx = df_lookup.columns.get_loc('prod_groupe') + 1
        prod_groupe_col_letter = get_column_letter(prod_groupe_col_idx)
        tunnel_col_idx = df_lookup.columns.get_loc('tunnel') + 1
        tunnel_col_letter = get_column_letter(tunnel_col_idx)

        # Plage des données dans la feuille pivot pour les formules
        pivot_data_start_row = 2
        pivot_data_start_col = 3
        pivot_data_end_row = sheet_pivot.max_row
        pivot_data_end_col = sheet_pivot.max_column
        pivot_data_range = f"'{pivot_sheet_name}'!${get_column_letter(pivot_data_start_col)}${pivot_data_start_row}:${get_column_letter(pivot_data_end_col)}${pivot_data_end_row}"
        pivot_row_header_range = f"'{pivot_sheet_name}'!$B${pivot_data_start_row}:$B${pivot_data_end_row}"
        pivot_col_header_range = f"'{pivot_sheet_name}'!${get_column_letter(pivot_data_start_col)}$1:${get_column_letter(pivot_data_end_col)}$1"

        # Remplissage des formules NombreEquip dans la feuille produits_tunnels
        for row_idx in range(2, sheet_lookup.max_row + 1):
            nombre_cell = sheet_lookup.cell(row=row_idx, column=nombre_equip_col_idx)
            equip_value = sheet_lookup[f'{equip_col_letter}{row_idx}'].value
            if (nombre_cell.value is None or nombre_cell.value == '') and equip_value and str(equip_value).strip() != '':
                eq_cell = f'{equip_col_letter}{row_idx}'
                tun_cell = f'{tunnel_col_letter}{row_idx}'
                formula = (
                    f'=IFERROR(INDEX({pivot_data_range},MATCH({eq_cell},{pivot_row_header_range},0),MATCH({tun_cell},{pivot_col_header_range},0)),"")'
                )
                nombre_cell.value = formula

        # --- Création des feuilles tunnels ---

        # Styles communs
        header_fill = PatternFill(start_color="FFF3E0", end_color="FFF3E0", fill_type="solid")
        thin = Side(border_style="thin", color="000000")
        all_border = Border(left=thin, right=thin, top=thin, bottom=thin)
        vertical_border = Border(left=thin, right=thin)

        # Groupes de tunnels pour couleur onglet
        green_tabs = {'Boissy', 'Champigny', 'Guy Môquet', 'Moulin', 'Nogent'}
        yellow_tabs = {'Ambroise Paré', 'Belle-Rive', 'Chennevières', 'Fontenay', 'La Défense',
                       'Nanterre Centre', 'Nanterre échangeur', 'Neuilly', 'Saint-Cloud', 'Sévines'}
        grey_tabs = {'Bobigny', 'La Courneuve', 'Landy', 'Lumen-Norton', 'Taverny'}
        tab_colors = {
            'green': 'C6EFCE',
            'yellow': 'FFF9C4',
            'grey': 'E0E0E0',
            'blue': 'DCE6F1'
        }

        unique_tunnels = df_lookup['tunnel'].dropna().unique()

        for tunnel in unique_tunnels:
            # Filtrer données pour le tunnel courant
            tunnel_df = df_lookup[df_lookup['tunnel'] == tunnel].drop_duplicates(subset=['mar_diminutif', 'prod_groupe'])

            # Colonnes à écrire dans la feuille tunnel
            sheet_df = tunnel_df[['mar_diminutif', 'prod_groupe', 'freq_annuelle',
                                  'ajust_freq_suppl', 'frq_totale', 'cout_total']].copy()
            sheet_df.columns = ['Marché', 'Opération', 'Fréquence annuelle',
                                'Ajustement fréquence supplémentaire', 'Fréquence totale',
                                'Coût total HT']
            sheet_df = sheet_df.reset_index(drop=True)
            # Ajouter colonnes vides "Nombre équipé" et "Coût unitaire HT"
            sheet_df.insert(5, 'Nombre équipé', '')
            sheet_df.insert(6, 'Coût unitaire HT', '')

            # Nom safe de la feuille
            safe_sheet_name = str(tunnel)[:31].replace('/', '-').replace('\\', '-')
            if tunnel in green_tabs:
                group = 'green'
            elif tunnel in yellow_tabs:
                group = 'yellow'
            elif tunnel in grey_tabs:
                group = 'grey'
            else:
                group = 'blue'

            # Création ou nettoyage de la feuille
            if safe_sheet_name in workbook.sheetnames:
                ws = workbook[safe_sheet_name]
                ws.delete_rows(2, ws.max_row)
            else:
                ws = workbook.create_sheet(title=safe_sheet_name)

            # Ecriture des entêtes (headers)
            headers = list(sheet_df.columns)
            for col_idx, col_name in enumerate(headers, 1):
                cell = ws.cell(row=1, column=col_idx, value=col_name)
                if col_name != 'Marché':
                    cell.font = Font(bold=True)
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.fill = header_fill
                cell.border = all_border

            max_row = len(sheet_df) + 1
            max_col = len(headers)

            # Ecriture des données + formules
            for i, row in enumerate(sheet_df.itertuples(index=False), start=2):
                for j, value in enumerate(row, start=1):
                    cell = ws.cell(row=i, column=j)
                    col_name = headers[j - 1]

                    # Formules spécifiques
                    if col_name == 'Nombre équipé':
                        formula = (
                            f'=IFERROR('
                            f'INDEX(\'{lookup_sheet_name}\'!${nombre_equip_col_letter}$2:${nombre_equip_col_letter}${sheet_lookup.max_row},'
                            f'MATCH(LOWER(B{i}) & "|{tunnel}",'
                            f'\'{lookup_sheet_name}\'!${key_col_letter}$2:${key_col_letter}${sheet_lookup.max_row},0)'
                            f'),"")'
                        )
                        cell.value = formula
                        cell.alignment = Alignment(horizontal="center", vertical="center")
                    elif col_name == 'Coût unitaire HT':
                        formula = (
                            f'=SUMIFS(\'{lookup_sheet_name}\'!${cout_ht_col_letter}$2:${cout_ht_col_letter}${sheet_lookup.max_row},'
                            f'\'{lookup_sheet_name}\'!${prod_groupe_col_letter}$2:${prod_groupe_col_letter}${sheet_lookup.max_row},LOWER(B{i}),'
                            f'\'{lookup_sheet_name}\'!${tunnel_col_letter}$2:${tunnel_col_letter}${sheet_lookup.max_row},"{tunnel}")'
                        )
                        cell.value = formula
                        cell.number_format = '#,##0.00'
                        cell.alignment = Alignment(horizontal="right", vertical="center")
                    elif col_name == 'Fréquence totale':
                        # Somme des 2 colonnes précédentes (Fréquence annuelle + Ajustement fréquence supplémentaire)
                        formula = f'=C{i} + D{i}'
                        cell.value = formula
                        cell.alignment = Alignment(horizontal="center", vertical="center")
                    elif col_name == 'Coût total HT':
                        # Multiplication Fréquence totale * Nombre équipé * Coût unitaire HT
                        formula = f'=E{i} * F{i} * G{i}'
                        cell.value = formula
                        cell.number_format = '#,##0.00'
                        cell.alignment = Alignment(horizontal="right", vertical="center")
                    else:
                        cell.value = value
                        if isinstance(value, (int, float)):
                            cell.number_format = '#,##0'
                            cell.alignment = Alignment(horizontal="center", vertical="center")

                    cell.border = vertical_border

            # Ligne total (somme Coût total HT)
            total_row = max_row + 1
            ws.cell(row=total_row, column=1, value="Total")
            ws.cell(row=total_row, column=1).font = Font(bold=True)
            ws.cell(row=total_row, column=1).alignment = Alignment(horizontal="right", vertical="center")

            sum_formula = f'=SUM(H2:H{max_row})'  # colonne H = Coût total HT (8ème colonne)
            total_sum_cell = ws.cell(row=total_row, column=8, value=sum_formula)
            total_sum_cell.font = Font(bold=True)
            total_sum_cell.number_format = '#,##0.00'
            total_sum_cell.alignment = Alignment(horizontal="right", vertical="center")

            # Ajuster la largeur des colonnes
            for col_idx in range(1, max_col + 1):
                col_letter = get_column_letter(col_idx)
                max_length = len(headers[col_idx - 1])

                for row_i in range(2, total_row + 1):
                    cell = ws.cell(row=row_i, column=col_idx)
                    val = cell.value
                    if val is None:
                        continue
                    if isinstance(val, (int, float)):
                        # Formater nombre avec deux décimales et virgule
                        val_str = f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
                    else:
                        val_str = str(val)
                    if len(val_str) > max_length:
                        max_length = len(val_str)

                # Colonne "Nombre équipé" limitée à largeur max 12
                if headers[col_idx - 1] == 'Nombre équipé':
                    ws.column_dimensions[col_letter].width = min(max_length + 2, 12)
                else:
                    ws.column_dimensions[col_letter].width = max_length + 2

            # Couleur de l'onglet selon le groupe
            tab_color = tab_colors[group]
            ws.sheet_properties.tabColor = tab_color

        # Ajuster la largeur des colonnes dans la feuille pivot aussi
        for col_idx in range(1, sheet_pivot.max_column + 1):
            col_letter = get_column_letter(col_idx)
            max_length = 0
            header = sheet_pivot.cell(row=1, column=col_idx).value
            if header:
                max_length = len(str(header))
            for row_i in range(2, sheet_pivot.max_row + 1):
                val = sheet_pivot.cell(row=row_i, column=col_idx).value
                if val is None:
                    continue
                val_str = str(val)
                if len(val_str) > max_length:
                    max_length = len(val_str)
            sheet_pivot.column_dimensions[col_letter].width = max_length + 2

        # Appliquer un format uniforme aux colonnes cout_ht dans produits_tunnels
        cout_ht_col_idx = df_lookup.columns.get_loc('cout_ht') + 1
        for row_i in range(2, sheet_lookup.max_row + 1):
            cell = sheet_lookup.cell(row=row_i, column=cout_ht_col_idx)
            if isinstance(cell.value, (int, float)):
                cell.number_format = '#,##0.00'

        # Ajuster largeur colonnes dans produits_tunnels
        for col_idx in range(1, sheet_lookup.max_column + 1):
            col_letter = get_column_letter(col_idx)
            max_length = 0
            header = sheet_lookup.cell(row=1, column=col_idx).value
            if header:
                max_length = len(str(header))
            for row_i in range(2, sheet_lookup.max_row + 1):
                val = sheet_lookup.cell(row=row_i, column=col_idx).value
                if val is None:
                    continue
                val_str = str(val)
                if len(val_str) > max_length:
                    max_length = len(val_str)
            # Colonne NombreEquip limitée largeur max 12
            if sheet_lookup.cell(row=1, column=col_idx).value == 'NombreEquip':
                sheet_lookup.column_dimensions[col_letter].width = min(max_length + 2, 12)
            else:
                sheet_lookup.column_dimensions[col_letter].width = max_length + 2

        writer.save()

# Exemple d'utilisation
create_combined_excel('equipements_tunnel.csv', 'produits_tunnels.csv', 'rapport_tunnels.xlsx')


/var/folders/s1/1wg2rzvx4_ndhknnyblvg_j80000gn/T/ipykernel_3354/3036152950.py:278: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
